# Get started with LangChain

## Table of Content

- Installation
- Set-up OpenAI Key
- Langchain Components
  - Model
  - Prompt Template
  - Output Parser
  - Chain
  - Memory
  - Agents
  - Loaders

> Note: Always a best practice is to change the Runtime
>
> Runtime -> Change Runtime type -> Select T4 GPU hardware accelerator

## Installation

In [2]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.3 MB/s eta 0:00:00


## Set-up OpenAI Key

In [3]:
import os
import langchain
import openai

In [4]:
openai_key = "sk-abc" #replace the key with your OpenAI Key
os.environ['OPENAI_API_KEY'] = openai_key
openai.api_key = os.environ['OPENAI_API_KEY']

## Model

Chat models are a variation on language models. While chat models use language models under the hood, the interface they expose is a bit different. Rather than expose a "text in, text out" API, they expose an interface where "chat messages" are the inputs and outputs.

![langchain-demo](https://python.langchain.com/assets/images/model_io-1f23a36233d7731e93576d6885da2750.jpg)

> Note: Large Language Models (LLMs) are a core component of LangChain. LangChain does not serve its own LLMs, but rather provides a standard interface for interacting with many different LLMs.

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

## LLMs

Large Language Models (LLMs) are the models that take a text string as input, and return a text string as output.

## Chat Models

Chat Models are the models that are usually backed by a language model, but their APIs are more structured. Specifically, these models take a list of Chat Messages as input, and return a Chat Message.

In [6]:
# To control the randomness and creativity of the generated, use temperature = 0.0 to 0.9
chat = ChatOpenAI(temperature=0.0,max_tokens = 500)
#chat

In [7]:
chat.model_name

'gpt-3.5-turbo'

In [8]:
print(chat.max_tokens)

500


In [9]:
llm = OpenAI(temperature=0.0)
#llm

In [10]:
llm.model_name

'text-davinci-003'

In [11]:
llm.max_tokens

256

## Prompt Template

LangChain offers tools for crafting and utilizing prompt templates, which are pre-defined guidelines for generating prompts for language models, including instructions, few-shot examples, and context-specific questions suitable for various tasks. These templates aim to be model-agnostic, allowing for easy reuse across different language models. Generally, language models anticipate prompts to be either a string or a series of chat messages."

In [12]:
from langchain.prompts import ChatPromptTemplate

> Let's create a prompt template, notice the variables inside {} curly brackets are treated as input variables and can be overriden based on the user input

## Please play around with your own prompts to understand how Prompt template works

In [13]:
template = """Generate a report \
highlighting the key findings from the {analysis_type} analysis. \
Here's a snippet of the analysis: \
```{analysis_snippet}```.
"""

In [14]:
prompt_temp = ChatPromptTemplate.from_template(template)

In [15]:
prompt_temp

ChatPromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], output_parser=None, partial_variables={}, template="Generate a report highlighting the key findings from the {analysis_type} analysis. Here's a snippet of the analysis: ```{analysis_snippet}```.\n", template_format='f-string', validate_template=True), additional_kwargs={})])

In [16]:
prompt_temp.input_variables

['analysis_snippet', 'analysis_type']

In [17]:
prompt_temp.messages

[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['analysis_snippet', 'analysis_type'], output_parser=None, partial_variables={}, template="Generate a report highlighting the key findings from the {analysis_type} analysis. Here's a snippet of the analysis: ```{analysis_snippet}```.\n", template_format='f-string', validate_template=True), additional_kwargs={})]

In [18]:
sample_analysis_type = "user_engagement"

sample_analysis_snippet = """
Over the past quarter, the user engagement metrics have shown promising growth.
The number of active users has increased by 15%, with a notable uptick in daily logins.
This increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.
Additionally, user interactions with key features, such as commenting and sharing, have risen by 20%.
This signifies a higher level of user participation and social engagement within our platform.
We've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.
The engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.
This suggests that our efforts to improve user experience and introduce new features are resonating well with our audience.

Overall, the data suggests that our focus on enhancing user engagement through product improvements and feature launches is yielding positive results. As we continue to refine our strategies and innovate, we can expect further growth in user interactions and overall engagement.
"""

> Using **format_messages** function we can override the input variables used in the Prompt template

In [19]:
response_message = prompt_temp.format_messages(
    analysis_type =sample_analysis_type,
    analysis_snippet = sample_analysis_snippet
)

In [20]:
response_message

[HumanMessage(content="Generate a report highlighting the key findings from the user_engagement analysis. Here's a snippet of the analysis: ```\nOver the past quarter, the user engagement metrics have shown promising growth.\nThe number of active users has increased by 15%, with a notable uptick in daily logins.\nThis increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.\nAdditionally, user interactions with key features, such as commenting and sharing, have risen by 20%.\nThis signifies a higher level of user participation and social engagement within our platform.\nWe've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.\nThe engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.\nThis suggests that our efforts to improve user experience and introduce new fe

In [21]:
response_message[0].content

"Generate a report highlighting the key findings from the user_engagement analysis. Here's a snippet of the analysis: ```\nOver the past quarter, the user engagement metrics have shown promising growth.\nThe number of active users has increased by 15%, with a notable uptick in daily logins.\nThis increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users.\nAdditionally, user interactions with key features, such as commenting and sharing, have risen by 20%.\nThis signifies a higher level of user participation and social engagement within our platform.\nWe've also observed longer average session durations, indicating that users are spending more time exploring the content we offer.\nThe engagement rate, calculated by dividing interactions by active users, has experienced a steady increase of 10% over the past two months.\nThis suggests that our efforts to improve user experience and introduce new features are resonating 

In [22]:
report = chat(response_message) #chatopenai model-GPT3.5 turbo

In [23]:
report_llm = llm(response_message[0].content) #OpenAI -llms - Text davinci-003

In [24]:
print(report.content)

User Engagement Analysis Report

Key Findings:

1. Active Users: The number of active users has increased by 15% over the past quarter. This growth can be attributed to the successful launch of our new mobile app, which has received positive feedback from users. The increase in active users indicates a growing user base and interest in our platform.

2. Daily Logins: There has been a notable uptick in daily logins, which further supports the increase in active users. This suggests that users are finding value in our platform and are regularly accessing it to engage with the content.

3. User Interactions: User interactions with key features, such as commenting and sharing, have risen by 20%. This signifies a higher level of user participation and social engagement within our platform. The increase in user interactions indicates that users are actively engaging with the content and finding it valuable enough to share and comment on.

4. Average Session Durations: We have observed longer

In [25]:
print(report_llm)


Report Summary:

The user engagement metrics for the past quarter have shown promising growth. The number of active users has increased by 15%, with a notable uptick in daily logins. This increase is largely attributed to the successful launch of our new mobile app, which has received positive feedback from users. Additionally, user interactions with key features, such as commenting and sharing, have risen by 20%. This signifies a higher level of user participation and social engagement within our platform. We've also observed longer average session durations, indicating that users are spending more time exploring the content we offer. The engagement rate has experienced a steady increase of 10% over the past two months.

Overall, the data suggests that our focus on enhancing user engagement through product improvements and feature launches is yielding positive results. As we continue to refine our strategies and innovate, we can expect further growth in user interactions and overall 

## Output in specific format and not string

When developing an application, there may be a requirement for the response to adhere to a specific format, such as JSON or XML.

In [26]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}
"""


In [27]:
movie_bot = ChatPromptTemplate.from_template(movie_review_template)
movie_bot

ChatPromptTemplate(input_variables=['review_text'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['review_text'], output_parser=None, partial_variables={}, template="For the following movie review, extract the following information:\n\ngenre: Mentioned genres in the review. Extract and list them as a comma separated Python list.\n\nlead_chemistry: Describe the chemistry between the lead characters, if mentioned.\n\nplot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.\n\nruntime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.\n\nFormat the output as JSON with the following keys:\ngenre\nlead_chemistry\nplot_twists\nruntime_feel\n\nreview_text: {review_text}\n", template_format='f-string', validate_template=True), additional_kwargs={})])

In [28]:
movie_bot.input_variables

['review_text']

In [29]:
movie_review = """\
This movie was an incredible cinematic experience. It offers a unique blend of genres:\
romance, drama, and thrilling action. The storyline is gripping, and the characters'\
emotional journey kept me engaged throughout. The visuals are stunning, especially\
during the breathtaking action sequences.

The acting was exceptional, with the lead actors delivering performances that truly\
captured the essence of their roles. The chemistry between the main characters added\
depth to the romantic elements of the plot. The supporting cast also deserves praise,\
as they brought their characters to life convincingly.

I was pleasantly surprised by the unexpected plot twists that kept me on the edge of\
my seat. The pacing was well-managed, allowing for a perfect balance between intense\
action and quieter, reflective moments.

While the movie was a bit longer than usual, I found myself immersed in the story\
and didn't mind the extended runtime. It's a film that leaves a lasting impression,\
prompting viewers to reflect on its themes long after the credits roll.
"""

In [31]:
delegate_review = movie_bot.format_messages(
    review_text = movie_review
)

In [32]:
delegate_review

[HumanMessage(content="For the following movie review, extract the following information:\n\ngenre: Mentioned genres in the review. Extract and list them as a comma separated Python list.\n\nlead_chemistry: Describe the chemistry between the lead characters, if mentioned.\n\nplot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.\n\nruntime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.\n\nFormat the output as JSON with the following keys:\ngenre\nlead_chemistry\nplot_twists\nruntime_feel\n\nreview_text: This movie was an incredible cinematic experience. It offers a unique blend of genres:romance, drama, and thrilling action. The storyline is gripping, and the characters'emotional journey kept me engaged throughout. The visuals are stunning, especiallyduring the breathtaking action sequences.\n\nThe acting was exceptional, with the lead actors delivering performances that tr

In [33]:
get_movie_details = chat(delegate_review)

In [34]:
print(get_movie_details.content)

{
  "genre": ["romance", "drama", "thrilling action"],
  "lead_chemistry": "The chemistry between the main characters added depth to the romantic elements of the plot.",
  "plot_twists": true,
  "runtime_feel": "I found myself immersed in the story and didn't mind the extended runtime."
}


As you can see the output is in the json format but the data type is still string

In [35]:
type(get_movie_details.content)

str

> In the code below, the .get command is used with JSON to retrieve details. However, since our response is in string format, this will result in an error.

In [36]:
get_movie_details.get("genre")

AttributeError: ignored

## Output Parser

Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

- "Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
- "Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.


In [37]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

 StructuredOutputParser output parser can be used when you want to return multiple fields. While the Pydantic/JSON parser is more powerful, we initially experimented with data structures having text fields only.

 To create such multiple data structures we use ResponseSchema

In [39]:
genre_schema = ResponseSchema(name="genre",
                              description="Mentioned genres in the review. \
                              Extract and list them as a comma separated Python list.")

lead_chemistry_schema = ResponseSchema(name="lead_chemistry",
                                       description="Describe the chemistry between \
                                       the lead characters, if mentioned.")

plot_twists_schema = ResponseSchema(name="plot_twists",
                                    description="Were any unexpected plot twists mentioned? \
                                    Answer True if yes, False if not or unknown.")

runtime_feel_schema = ResponseSchema(name="runtime_feel",
                                     description="Did the reviewer mention their feelings about \
                                     the movie's runtime? If yes, extract their sentiment about it.")

response_schemas = [genre_schema,
                    lead_chemistry_schema,
                    plot_twists_schema,
                    runtime_feel_schema]

In [40]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [41]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"genre": string  // Mentioned genres in the review.                               Extract and list them as a comma separated Python list.
	"lead_chemistry": string  // Describe the chemistry between                                        the lead characters, if mentioned.
	"plot_twists": string  // Were any unexpected plot twists mentioned?                                     Answer True if yes, False if not or unknown.
	"runtime_feel": string  // Did the reviewer mention their feelings about                                      the movie's runtime? If yes, extract their sentiment about it.
}
```


> Note: Use this format_instructions in your prompt template to get the response in JSON

In [42]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}

{format_instructions}
"""


In [43]:
prompt = ChatPromptTemplate.from_template(template=movie_review_template)

In [57]:
user_review = prompt.format_messages(
    review_text=movie_review,
    format_instructions=format_instructions
)

In [58]:
response = chat(user_review)

In [59]:
print(response.content)

```json
{
	"genre": ["romance", "drama", "thrilling action"],
	"lead_chemistry": "The chemistry between the main characters added depth to the romantic elements of the plot.",
	"plot_twists": true,
	"runtime_feel": "While the movie was a bit longer than usual, I found myself immersed in the story and didn't mind the extended runtime."
}
```


In [60]:
final_ans = output_parser.parse(response.content)

In [61]:
final_ans

{'genre': ['romance', 'drama', 'thrilling action'],
 'lead_chemistry': 'The chemistry between the main characters added depth to the romantic elements of the plot.',
 'plot_twists': True,
 'runtime_feel': "While the movie was a bit longer than usual, I found myself immersed in the story and didn't mind the extended runtime."}

In [62]:
final_ans.get("genre")

['romance', 'drama', 'thrilling action']

## Chain

In [63]:
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [64]:
movie_review_template = """\
For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: {review_text}
"""

In [65]:
user_prompt = ChatPromptTemplate.from_template(movie_review_template)

In [66]:
chat = ChatOpenAI(temperature=0.0) #ChatOPENAI

## LLM Chain

> Note:
>
> verbose = True, shows the logs that happens in the background
>
> verbose = False, displays no logs

In [67]:
chain = LLMChain(llm=chat,prompt=user_prompt,verbose=True) # by default - Verbose=False
review_res = chain.run(movie_review)



> Entering new LLMChain chain...
Prompt after formatting:
Human: For the following movie review, extract the following information:

genre: Mentioned genres in the review. Extract and list them as a comma separated Python list.

lead_chemistry: Describe the chemistry between the lead characters, if mentioned.

plot_twists: Were any unexpected plot twists mentioned? Answer True if yes, False if not or unknown.

runtime_feel: Did the reviewer mention their feelings about the movie's runtime? If yes, extract their sentiment about it.

Format the output as JSON with the following keys:
genre
lead_chemistry
plot_twists
runtime_feel

review_text: This movie was an incredible cinematic experience. It offers a unique blend of genres:romance, drama, and thrilling action. The storyline is gripping, and the characters'emotional journey kept me engaged throughout. The visuals are stunning, especiallyduring the breathtaking action sequences.

The acting was exceptional, with the lead actors deliv

In [75]:
print(review_res)

{
  "genre": ["romance", "drama", "thrilling action"],
  "lead_chemistry": "The chemistry between the main characters added depth to the romantic elements of the plot.",
  "plot_twists": true,
  "runtime_feel": "I found myself immersed in the story and didn't mind the extended runtime."
}


## SequentialChain

In [76]:
chain_one = LLMChain(llm=chat,prompt=user_prompt,output_key = "json_answer") #outputkey can be any name

In [77]:
summary_review = "Please summarize the following review in 1 liner: {review_text}"
second_prompt_template = ChatPromptTemplate.from_template(summary_review)

In [78]:
chain_two = LLMChain(llm =chat,prompt=second_prompt_template,output_key = "summary")

In [79]:
seq_chain = SequentialChain(
    chains=[chain_one, chain_two],
    input_variables=["review_text"],
    output_variables=["json_answer", "summary"],
    verbose=True
)

In [80]:
seq_response = seq_chain(movie_review)



> Entering new SequentialChain chain...

> Finished chain.


In [81]:
seq_response

{'review_text': "This movie was an incredible cinematic experience. It offers a unique blend of genres:romance, drama, and thrilling action. The storyline is gripping, and the characters'emotional journey kept me engaged throughout. The visuals are stunning, especiallyduring the breathtaking action sequences.\n\nThe acting was exceptional, with the lead actors delivering performances that trulycaptured the essence of their roles. The chemistry between the main characters addeddepth to the romantic elements of the plot. The supporting cast also deserves praise,as they brought their characters to life convincingly.\n\nI was pleasantly surprised by the unexpected plot twists that kept me on the edge ofmy seat. The pacing was well-managed, allowing for a perfect balance between intenseaction and quieter, reflective moments.\n\nWhile the movie was a bit longer than usual, I found myself immersed in the storyand didn't mind the extended runtime. It's a film that leaves a lasting impression,p

## Memory

![lanchain_memory](https://python.langchain.com/assets/images/memory_diagram-0627c68230aa438f9b5419064d63cbbc.png)

In [82]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [83]:
temp1 = ChatPromptTemplate.from_template("Hi, my name is {name}, I work at AI Planet")
temp1 = temp1.format_messages(name="Tarun Jain")
res = chat(temp1)

In [84]:
res

AIMessage(content='Hello Tarun Jain! Nice to meet you. How can I assist you today?', additional_kwargs={}, example=False)

In [85]:
temp2 = ChatPromptTemplate.from_template("As you know I work at {startup}. What is my name?")
temp2 = temp2.format_messages(startup="AI Planet")
res2 = chat(temp2)

In [86]:
res2

AIMessage(content="I'm sorry, but as an AI language model, I don't have access to personal information about individuals unless it has been shared with me in the course of our conversation. I am designed to respect user privacy and confidentiality. Therefore, I don't know your name unless you explicitly tell me. My primary function is to provide information and answer questions to the best of my knowledge and abilities. If you have any concerns about privacy or data security, please let me know, and I will do my best to address them.", additional_kwargs={}, example=False)

## Memory

Most LLM applications have a conversational interface. An essential component of a conversation is being able to refer to information introduced earlier in the conversation. At bare minimum, a conversational system should be able to access some window of past messages directly. A more complex system will need to have a world model that it is constantly updating, which allows it to do things like maintain information about entities and their relationships.

In [91]:
memory = ConversationBufferMemory()

In [92]:
conversation = ConversationChain(
    llm=chat,
    memory = memory,
    verbose=True
)

In [93]:
conversation.predict(input="Hi, my name is Tarun Jain, I work at AI Planet")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Tarun Jain, I work at AI Planet
AI:

> Finished chain.


"Hello Tarun Jain! It's nice to meet you. I'm an AI and I'm here to chat with you. How can I assist you today?"

In [94]:
conversation.predict(input="At AI Planet I work as a DevRel & Community Manager")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Tarun Jain, I work at AI Planet
AI: Hello Tarun Jain! It's nice to meet you. I'm an AI and I'm here to chat with you. How can I assist you today?
Human: At AI Planet I work as a DevRel & Community Manager
AI:

> Finished chain.


"That's great! As a DevRel & Community Manager, your role is to build and maintain relationships with developers and the community. You likely organize events, create content, and provide support to developers using AI technologies. It's an important role in fostering a strong developer community and driving adoption of AI solutions. Is there anything specific you would like to discuss or any questions you have about your role?"

In [95]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Tarun Jain, I work at AI Planet
AI: Hello Tarun Jain! It's nice to meet you. I'm an AI and I'm here to chat with you. How can I assist you today?
Human: At AI Planet I work as a DevRel & Community Manager
AI: That's great! As a DevRel & Community Manager, your role is to build and maintain relationships with developers and the community. You likely organize events, create content, and provide support to developers using AI technologies. It's an important role in fostering a strong developer community and driving adoption of AI solutions. Is there anything specific you would like to discuss or any questions you have about your role?
Human: What is 

'Your name is Tarun Jain.'

In [96]:
print(memory.buffer)

Human: Hi, my name is Tarun Jain, I work at AI Planet
AI: Hello Tarun Jain! It's nice to meet you. I'm an AI and I'm here to chat with you. How can I assist you today?
Human: At AI Planet I work as a DevRel & Community Manager
AI: That's great! As a DevRel & Community Manager, your role is to build and maintain relationships with developers and the community. You likely organize events, create content, and provide support to developers using AI technologies. It's an important role in fostering a strong developer community and driving adoption of AI solutions. Is there anything specific you would like to discuss or any questions you have about your role?
Human: What is my name?
AI: Your name is Tarun Jain.


## Agents

Some applications will require not just a predetermined chain of calls to LLMs/other tools, but potentially an unknown chain that depends on the user's input. In these types of chains, there is a “agent” which has access to a suite of tools. Depending on the user input, the agent can then decide which, if any, of these tools to call.

In [102]:
!pip install duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [99]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.utilities import DuckDuckGoSearchAPIWrapper
from langchain.agents import initialize_agent

In [103]:
search = DuckDuckGoSearchAPIWrapper()
tools = [
    Tool(
        name = "Search Engine",
        func=search.run,
        description="To explore the world of internet"
    ),
]

In [104]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
llm = ChatOpenAI(openai_api_key=openai_key, temperature=0.0)
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

### Agent Type: Do refer the official documentation [docs](https://python.langchain.com/docs/modules/agents/agent_types/)


In [105]:
agent_chain.run(input="Where is Bangalore located?")



> Entering new AgentExecutor chain...
{
    "action": "Search Engine",
    "action_input": "Location of Bangalore"
}
Observation: It lies 3,113 feet (949 metres) above sea level, atop an east-west ridge in the Karnataka Plateau in the southeastern part of the state, at a cultural meeting point of the Kannada -, Telugu -, and Tamil -speaking peoples. Pop. (2001) city, 4,301,326; urban agglom., 5,701,446; (2011) city, 8,443,675; urban agglom., 8,520,435. History MapsofIndia.com - Map showing the location of Bangalore,Karnataka in India. Find where is Bangalore located. Bengaluru Railway Map. Bengaluru or Bangalore is the capital of Karnataka and one of the most prominent cities of India. It has a population of 8,443,675 (2011 census) and ranks as the third ... About Map: Map showing location of Bangalore in the state of Karnatak, India Where is Bengaluru Located? Bangalore officially Bengaluru is the capital and largest city of the Indian state of Karnataka. Bengaluru lies between 12°5

'Bangalore is located in the southeastern part of the state of Karnataka, India. It is situated atop an east-west ridge in the Karnataka Plateau, at an elevation of 3,113 feet (949 meters) above sea level. Bangalore is a cultural meeting point for the Kannada, Telugu, and Tamil-speaking peoples.'

## Reference:

- [AIWithTarun Video](https://www.youtube.com/watch?v=IbMtSXTJ0ic)
- [Langchain documentation](https://docs.langchain.com/docs/)